한국어 Word2Vec 만들기(네이버 뉴스)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import re
from eunjeon import Mecab

기사 데이터 불러오기

In [9]:
상폐기업 = pd.read_csv('./상폐_새로합친거.csv',index_col=0)
상폐기업
# 정상기업 = pd.read_csv('./data/정상기업뉴스.csv')


,기업,기사발행일,기사제목,뉴스기사본문전처리
0,네이쳐글로벌,2010.10.01. 오전 9:23,"[특징주] 네이쳐글로벌 상한가…""전 대표 횡령 확인 안돼""",비메모리 반도체 업체인 네이쳐글로벌이 전 대표 횡령 혐의에 대해 확인된 바 없다고 ...
1,네이쳐글로벌,2010.09.28. 오후 4:29,"네이쳐글로벌, 네비스탁측이 74만여주 장내매도",네비스탁은 특별관계자로 포함된 개인투자자들이 네이쳐글로벌 주식 만주를 장내 매도했...
2,네이쳐글로벌,2010.09.16. 오전 9:23,"[특징주] 네이쳐글로벌, 투자경고 지정 이후 약세",네이쳐글로벌이 투자경고 지정 소식에 이어 전환사채 물량 상장에 따른 수급 불안으로 ...
3,네이쳐글로벌,2010.09.14. 오후 4:21,"네비스탁외 51인, 네이쳐글로벌 지분 1.26% 줄어",네비스탁외 인은 일 신규 소액주주 의결권 규합 및 보유주식수 변동에 따라 네이쳐글...
4,네이쳐글로벌,2010.09.16. 오후 8:08,"네이쳐글로벌, 공시번복..불성실공시법인 지정",한국거래소 코스닥시장본부는 일 네이쳐글로벌에 대해 공시번복으로 인해 불성실공시법인...
...,...,...,...,...
2696,코썬바이오,2020.04.24. 오후 2:17,"코썬바이오, 불성실 공시 끝에 결국 상장폐지",월일 시장위에서 최종 결정개미들의 억원 휴지조각 돼서울뉴시스 지난 년 코썬바이오의 ...
2697,코썬바이오,2020.01.09. 오전 10:07,"코썬바이오, 상한가 진입... 이평선 역배열 상황에서 반등 시도",코썬바이오일 오전 코썬바이오가 상한가를 기록했다. 코썬바이오는 균형생식환 황찬고 등...
2698,코썬바이오,2020.03.04. 오후 5:45,"코썬바이오, 거래소에 개선계획서 제출",한국거래소 코스닥시장본부는 일 상장적격성 실질심사 대상으로 결정된 코썬바이오가 개...
2699,코썬바이오,2020.01.10. 오후 6:42,코썬바이오 거래정지,한국거래소는 코스닥 상장기업인 코썬바이오에 대해 월 일부터 상장적격성 실질심사 대상...


단어가 폐지유사단어 들어가 있는 뉴스기사 뽑기

In [10]:
분류단어 = ['폐지', '격성', '퇴출', '실질', '심사', '속개', '이의', '심의', '거절']
분류단어 = '|'.join(분류단어)
분류단어

'폐지|격성|퇴출|실질|심사|속개|이의|심의|거절'

In [75]:
분류단어 = ['폐지', '격성', '퇴출', '실질', '심사', '속개', '이의', '심의', '거절']
분류단어 = '|'.join(분류단어)
분류단어

,기업,기사발행일,기사제목,뉴스기사본문전처리
9,네이쳐글로벌,2010.09.03. 오후 5:18,"네이쳐글로벌, 전환청구권 240만주 행사",네이쳐글로벌은 전환청구권 만주를 전환가액 원에 행사한다고 일 공시했다. 전체 물량의...
22,네이쳐글로벌,2010.04.13. 오전 7:56,"네이쳐글로벌, 네이처메이드 대상 80만주 유증",네이쳐글로벌은 제자인 네이쳐메이드와 남밤비를 대상으로 만주의 유상증자를 하기로 결의...
25,네이쳐글로벌,2010.02.01. 오후 5:32,"네이쳐글로벌, LED업체 인수…연 1천억 매출 기대",네이쳐글로벌이 일 일본의 업체 와 경영권 및 주식 양수도계약을 체결했다고 밝혔다....
30,네이쳐글로벌,2010.10.03. 오후 5:07,코스닥 불성실공시법인 큰폭 증가,하반기 들어 크게 줄어들었던 코스닥 기업의 불성실공시법인지정건수가 지난달 큰 폭으로...
33,네이쳐글로벌,2010.06.04. 오전 8:23,장 마감 후 주요 종목 뉴스-3일,코스피 시장남양개발은 계열회사인 부영주택으로부터 억만원을 장기차입하는 계약을 체결했...
...,...,...,...,...
2688,레이젠,2018.05.10. 오전 7:15,[e공시 눈에 띄네] 코스닥-9일,장 종료 후 공시캐스텍코리아 최대주주 변경을 수반하는 주식양수도 계약 체결수성 공...
2694,레이젠,2018.04.18. 오전 9:25,[e공시 눈에띄네]코스닥-17일,장 마감 후 주요 공시우리넷 최대주주 세티밸류업홀딩스로 변경거래소 나노메딕스에 주가...
2695,코썬바이오,2020.04.24. 오전 11:25,"코썬바이오(구 현성바이탈), 기심위서 '상장폐지' 결론",코썬바이오가 증시에서 퇴출될 위기에 처했다. 일 한국거래소에 따르면 코스닥시장본부...
2696,코썬바이오,2020.04.24. 오후 2:17,"코썬바이오, 불성실 공시 끝에 결국 상장폐지",월일 시장위에서 최종 결정개미들의 억원 휴지조각 돼서울뉴시스 지난 년 코썬바이오의 ...


뉴스기사 레이블 1인것 합친 수

In [39]:
data_df=train_data.groupby(["기업"]).sum()
data_df
# data_df["부도기사 비율"]=data_df["뉴스기사 레이블"]
len(data_df["뉴스기사 레이블"])

127

In [36]:
count_df=train_data.groupby(["기업"]).count()
count_df.columns=['기사발행일', '기사제목', '뉴스기사본문', '뉴스기사본문전처리', '뉴스기사 수']
count_df

,기사발행일,기사제목,뉴스기사본문,뉴스기사본문전처리,뉴스기사 수
기업,,,,,
KCW,14,14,14,14,14
글로웍스,24,24,24,24,24
나노트로닉스,10,10,10,10,10
나이스메탈,27,27,27,27,27
네스테크,29,29,29,29,29
...,...,...,...,...,...
한와이어리스,34,34,34,34,34
해피드림,28,28,28,28,28
핸디소프트,12,12,12,12,12


In [37]:
data_df["부도기사 비율"]=round(data_df["뉴스기사 레이블"]/count_df["뉴스기사 수"]*100,2)
final_df=pd.DataFrame(data_df["부도기사 비율"])
final_df.to_csv("상폐기업_부도기사 비율.csv",encoding="utf-8-sig")

들어간 기사 수 개수 세기

In [38]:
# plor=["폐지","퇴출","기로","벼랑"]
# a="|".join(plor)
# df=train_data[train_data['뉴스기사본문전처리'].str.contains(a)]
# df

s